Following the steps [here](https://haystack.deepset.ai/tutorials/07_rag_generator)

In [1]:
# Import some files
import pickle
with open('../data/chapter_fmt_list.pkl','rb') as f:
    chapters = pickle.load(f)

In [2]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


In [3]:
from haystack import Document
chapter_documents = [Document.from_dict(d) for d in chapters]

In [4]:
# Try with a few different Preprocessors on either passages (too long) or sentence. We'll probably need to tinker with this
from haystack.nodes import PreProcessor

word_preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
    split_overlap=20,
    progress_bar=True, 
    add_page_number=True
)

sentence_preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="sentence",
    split_length=6,
    split_respect_sentence_boundary=False,
    split_overlap=2,
    progress_bar=True, 
    add_page_number=True
)
# Should add max_chars_check or similar once we get to the point we do a dense retreival model

In [5]:
docs_word = word_preprocessor.process(chapter_documents)
docs_sentence = sentence_preprocessor.process(chapter_documents)

Preprocessing:   0%|          | 0/307 [00:00<?, ?docs/s]

WARNING - haystack.nodes.preprocessor.preprocessor -  We found one or more sentences whose word count is higher than the split length.


Preprocessing:   0%|          | 0/307 [00:00<?, ?docs/s]

In [6]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import RAGenerator, DensePassageRetriever


# Initialize FAISS document store.
# Set `return_embedding` to `True`, so generator doesn't have to perform re-embedding
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)

# Initialize DPR Retriever to encode documents, encode question and query documents
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=True,
    embed_title=True,
)

# Initialize RAG Generator
generator = RAGenerator(
    model_name_or_path="facebook/rag-token-nq",
    use_gpu=True,
    top_k=1,
    max_length=200,
    min_length=2,
    embed_title=True,
    num_beams=2,
)


INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1


INFO - haystack.modeling.model.language_model -  Auto-detected model language: english


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


INFO - haystack.modeling.model.language_model -  Auto-detected model language: english
INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1


/home/gibson/pale-companion/pale-companion/lib/python3.10/site-packages/transformers/models/bart/configuration_bart.py:179: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizerFast'.


Some weights of the model checkpoint at facebook/rag-token-nq were not used when initializing RagTokenForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.weight', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing RagTokenForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagTokenForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RagTokenForGeneration were not initialized from the model checkpoint at facebook/rag-token-nq and are newly initialized: ['rag.generator.lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictio

In [7]:
# Delete existing documents in documents store
document_store.delete_documents()

# Write documents to document store
document_store.write_documents(docs_word)

# Add documents embeddings to index
document_store.update_embeddings(retriever=retriever)


Writing Documents:   0%|          | 0/52859 [00:00<?, ?it/s]

In [ ]:
QUESTIONS = [
    "Who is Avery?",
    "Where does Verona live?",
    "What is Toadswallow?",
    "How many Carmine conspirators are there?",
    "What does Alpeona do?",
    "Why is Gilkey ostracized?",
    "What is Snowdrop?",
    "Why is Maricaca scared of?",
    "What are the opinions of Charles?",
    "How many others live in Kennet?",
    "Why was Seth Forsaken?",
    "Is Alexander dead?",
    "Where is Bristow?",
    "What is the Wolf?",
    "What is the Red Heron?",
    "Can we talk about the girls?",
    "Who has a crush on Avery?",
    "What are the most powerful Others?",
    "What are the boons for Sootsleeves path?"
]


In [ ]:
# Or alternatively use the Pipeline class
from haystack.pipelines import GenerativeQAPipeline
from haystack.utils import print_answers

pipe = GenerativeQAPipeline(generator=generator, retriever=retriever)
for question in QUESTIONS:
    res = pipe.run(query=question, params={"Generator": {"top_k": 2}, "Retriever": {"top_k": 5}})
    print_answers(res, details="minimum")
